In [2]:
from qconv import TBNConv2d
import torch
import torch.nn as nn
import torch.nn.functional as F

tbnconv = TBNConv2d(64, 2, 3, 1, 1, adc_mode=True)
inputs = torch.randn(1, 64, 4, 4)
tbnconv.pim_mode=True
tbnconv.adc_mode='linear'
tbnconv.adc_bits=5
linear_adc_out = tbnconv(inputs)
tbnconv.adc_mode='original'
original_out = tbnconv(inputs)
tbnconv.adc_mode='none'
out = tbnconv(inputs)
print("modified ADC out \n",linear_adc_out)
print("original ADC out \n", original_out)
print("Without ADC out \n", out)
print("modified adc diff : ", linear_adc_out - out)
print("original adc diff : ", original_out - out)
print("diff mse : ", F.mse_loss(linear_adc_out, out))
print("diff original_adc mse : ", F.mse_loss(original_out, out))


modified ADC out 
 tensor([[[[-0.5742, -0.4306, -0.7177, -0.7177],
          [-0.8612, -0.1435, -1.1483,  0.2871],
          [-0.5742, -1.0048, -0.7177, -0.7177],
          [-0.1435, -0.7177, -0.5742,  0.0000]],

         [[ 0.0000, -0.2871,  0.2871, -0.4306],
          [-0.1435, -0.5742, -1.0048, -0.7177],
          [-0.8612, -0.4306, -0.8612, -0.7177],
          [ 0.0000, -0.5742, -0.2871, -0.4306]]]], grad_fn=<AddBackward0>)
original ADC out 
 tensor([[[[-0.3577, -0.2534, -0.3701, -0.4631],
          [-0.4441,  0.3779, -0.8063,  0.6392],
          [-0.3134, -0.3375, -0.1996, -0.2938],
          [ 0.0678, -0.3858, -0.2271,  0.1015]],

         [[ 0.2131, -0.1699,  0.3695, -0.2047],
          [ 0.1817,  0.0768, -0.4345, -0.3291],
          [-0.4985,  0.1228, -0.1138, -0.2635],
          [ 0.1060, -0.2007,  0.0533, -0.1688]]]], grad_fn=<AddBackward0>)
Without ADC out 
 tensor([[[[-0.2871, -0.1794, -0.3230, -0.3947],
          [-0.3768,  0.4486, -0.7536,  0.6998],
          [-0.2512, -0